<a href="https://colab.research.google.com/github/mrcruz117/SARIMAX---Parameter-Tuning-Template/blob/main/SARIMAX_Parameter_Tuning_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries and Data

In [7]:
!pip install pmdarima

In [8]:
%cd /content/drive/MyDrive/Time\ Series\ Forecasting\ Product

/content/drive/MyDrive/Time Series Forecasting Product


In [9]:
#libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import ParameterGrid
import pmdarima as pm

In [10]:
# pmdarima dep fix
# 1. Blow away any existing installs
# !pip uninstall -y numpy

# 2. Install the NumPy version known to work (1.23.x series has ABI size 96)
# !pip install "numpy<2"



# 3. Force pmdarima to build its C extensions against that NumPy
# !pip install --no-binary :all: pmdarima


In [11]:
#load the data
#YYYY-MM-DD
df = pd.read_csv('nyc_data.csv', index_col = 0, parse_dates = True)
df.head()

,Demand,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,,
2015-01-01,720.000885,0,0,0,3.68,41.305
2015-01-02,581.276773,0,0,0,4.73,131.574
2015-01-03,754.117039,0,0,0,7.23,162.700
2015-01-04,622.252774,0,0,0,10.96,160.281
2015-01-05,785.373319,0,0,0,6.92,51.077


In [12]:
#Rename variable
df = df.rename(columns = {'Demand': 'y'})
df.head(0)

,y,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,,


In [13]:
# Extract Regressors
X = df.iloc[:,1:]
X.head(0)

,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,


# Stationarity

In [14]:
# Test
from statsmodels.tsa.stattools import adfuller

af = adfuller(x=df.y)

pvalue = af[1]

# Dickey–Fuller test
if pvalue < 0.05:
  print("Time Series is stationary")
  print(f"p-value: {pvalue}")
else:
  print("Time Series is NOT stationary")
  print(f"p-value: {pvalue}")


Time Series is NOT stationary
p-value: 0.37677707077291045


In [15]:
# differencing

af = adfuller(x=df.y.diff().dropna())
pvalue = af[1]

# Dickey–Fuller test
if pvalue < 0.05:
  print("Time Series is stationary")
  print(f"p-value: {pvalue}")
else:
  print("Time Series is NOT stationary")
  print(f"p-value: {pvalue}")

Time Series is stationary
p-value: 3.3557739456287946e-22


## SARIMAX Model

In [16]:
# model

model = pm.ARIMA(
    order=(1,1,1),
    seasonal_order=(1,1,1, 7),
    suppress_warnings=True,
    enforce_stationarity=False,
    )